# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Classificação-de-Textos" data-toc-modified-id="Classificação-de-Textos-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Classificação de Textos</a></div><div class="lev2 toc-item"><a href="#Preâmbulo" data-toc-modified-id="Preâmbulo-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Preâmbulo</a></div><div class="lev2 toc-item"><a href="#Preparando-o-dataset" data-toc-modified-id="Preparando-o-dataset-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Preparando o dataset</a></div><div class="lev3 toc-item"><a href="#Buscando-o-texto-dos-livros-e-definindo-os-rótulos" data-toc-modified-id="Buscando-o-texto-dos-livros-e-definindo-os-rótulos-121"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Buscando o texto dos livros e definindo os rótulos</a></div><div class="lev3 toc-item"><a href="#Representando-as-palavras-através-de-índices-inteiros" data-toc-modified-id="Representando-as-palavras-através-de-índices-inteiros-122"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Representando as palavras através de índices inteiros</a></div><div class="lev3 toc-item"><a href="#Palavras-características-de-cada-livro" data-toc-modified-id="Palavras-características-de-cada-livro-123"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Palavras características de cada livro</a></div><div class="lev2 toc-item"><a href="#Dividindo-o-dataset-entre-treinamento-e-validação" data-toc-modified-id="Dividindo-o-dataset-entre-treinamento-e-validação-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Dividindo o dataset entre treinamento e validação</a></div><div class="lev3 toc-item"><a href="#Divisão-simples" data-toc-modified-id="Divisão-simples-131"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Divisão simples</a></div><div class="lev3 toc-item"><a href="#Divisão-para-uso-com-geradores-e-aumento-de-dados" data-toc-modified-id="Divisão-para-uso-com-geradores-e-aumento-de-dados-132"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Divisão para uso com geradores e aumento de dados</a></div><div class="lev3 toc-item"><a href="#Criando--geradores-para-treino-e-validação" data-toc-modified-id="Criando--geradores-para-treino-e-validação-133"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Criando  geradores para treino e validação</a></div>

# Classificação de Textos

## Preâmbulo

In [1]:
%matplotlib inline
import matplotlib.pyplot as plot
from IPython import display
from __future__ import print_function

import os
import sys
import glob
import numpy as np
import numpy.random as nr

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.optimizers import (SGD, 
                              RMSprop, 
                              Adam, 
                              Adadelta, 
                              Adagrad)

sys.path.append('../src')
from my_keras_utilities import (get_available_gpus, 
                                load_model_and_history, 
                                save_model_and_history, 
                                TrainingPlotter)

os.makedirs('../../models',exist_ok=True)
np.set_printoptions(precision=3, linewidth=120)

Using TensorFlow backend.


In [2]:
import keras.backend as K

# K.set_image_data_format('channels_first')
K.set_floatx('float32')

print('Backend:        {}'.format(K.backend()))
print('Data format:    {}'.format(K.image_data_format()))
print('Available GPUS:', get_available_gpus())

Backend:        tensorflow
Data format:    channels_first
Available GPUS: []


## Preparando o dataset

### Buscando o texto dos livros e definindo os rótulos

In [3]:
data_dir = '../../datasets/'

autores = [
#     'Fernando_Sabino', 
    'Jorge_Amado',
    'Machado_de_Assis',
    'Érico_Veríssimo',
]

book_text = []
book_author = []
book_title = []
for aut in autores:
    for fn in glob.glob(data_dir + 'livros/' + aut + '*.txt'):
        author, book = os.path.basename(fn).split('__')
        txt = open(fn).read().replace('\x97', '')
        book_text.append(txt)
        book_author.append(author)
        book_title.append(book[:-4])
        print('{:7d}  {:18s} {}'.format(len(txt), author, book[:-4]))

author_list = list(set(book_author))
n_labels = len(author_list)
n_books = len(book_title)
book_label = [author_list.index(a) for a in book_author]
print('\n{} Labels:'.format(n_labels))
for i, autor in enumerate(author_list):
    print('    {:2d}: {}'.format(i, autor))

 427711  Jorge_Amado        Capitães_de_Areia
1030735  Jorge_Amado        Dona_flor_seus_dois_maridos
 828417  Jorge_Amado        Gabriela
1001226  Jorge_Amado        Tereza_Batista_Cansada_de_Guerra
 372459  Machado_de_Assis   Dom_Casmurro
 411043  Machado_de_Assis   Esaú_e_Jacó
 337533  Machado_de_Assis   Helena
 336677  Machado_de_Assis   Iaiá_Garcia
 280683  Machado_de_Assis   Memorial_de_Aires
 352965  Machado_de_Assis   Memórias_Póstumas_de_Brás_Cubas
 443778  Machado_de_Assis   Quincas_Borba
 294049  Érico_Veríssimo  Clarissa
 890215  Érico_Veríssimo  Incidente_em_Antares
 699390  Érico_Veríssimo  O_Tempo_e_o_Vento_-_O_Arquipélago
 749265  Érico_Veríssimo  O_Tempo_e_o_Vento_-_O_Continente

3 Labels:
     0: Machado_de_Assis
     1: Érico_Veríssimo
     2: Jorge_Amado


### Representando as palavras através de índices inteiros

In [4]:
MAX_NB_WORDS = 20000

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(book_text)
sequences = tokenizer.texts_to_sequences(book_text)

w2i = tokenizer.word_index
i2w = dict([(v, k) for k, v in w2i.items()])

i2w_vec = np.array([i2w[i+1] for i in range(len(i2w))])

print('Found %s unique tokens.' % len(w2i))

Found 61707 unique tokens.


In [5]:
for i, seq in enumerate(sequences):
    k = nr.randint(len(seq) - 100)
    print('{}: {} -- {} palavras'.format(book_author[i], book_title[i], len(seq)).replace('_', ' '))
    print(' '.join([i2w[x] for x in seq[k:k+100]]), end='\n\n')


Jorge Amado: Capitães de Areia -- 76132 palavras
aquele saber aquela vocação para contar histórias fizera o respeitado entre os capitães areia se bem fosse franzino magro e triste o cabelo moreno caindo sobre os olhos apertados de míope apelidaram no de professor porque num livro furtado ele aprendera a fazer mágicas com lenços níqueis e também porque contando aquelas histórias que lia e muitas que inventava fazia a grande e misteriosa mágica de os transportar para mundos diversos fazia com que o s olhos vivos dos capitães da areia como só brilham as estrelas da noite da bahia pedro bala nada resolvia sem o consultar e várias

Jorge Amado: Dona flor seus dois maridos -- 166625 palavras
muita competência com os olhos acesos completou vejam como vai se rebolando a cara séria mas as ancas olhem aquilo soltas até parece que algu ém está nelas um felizardo esse doutor do braço do marido felizardo sorri mansa dona flor ah essa mania de vadinho ir pela rua a lhe tocar os peitos e os quadris 

### Palavras características de cada livro

In [6]:
tfidf = tokenizer.sequences_to_matrix(sequences, mode='tfidf')
ww = np.argsort(tfidf, axis=1)[:, -8:]
print(i2w_vec[ww-1])

[['trapiche' 'bedel' 'ester' 'barandão' 'almiro' 'dora' '–' 'pirulito']
 ['marilda' 'dinorá' 'teodoro' 'pelancchi' 'gisa' 'mirandão' 'rozilda' 'vadinho']
 ['tuísca' 'tonico' 'ribeirinho' 'malvina' 'amâncio' 'fulgêncio' 'gabriela' 'nacib']
 ['vavá' 'almério' 'januário' '—' 'dóris' 'brígida' 'justiniano' 'tereza']
 ['manduca' 'protonotário' 'bentinho' 'sancha' 'pádua' 'justina' 'escobar' 'capitu']
 ['coupé' 'gêmeos' 'excia' 'nóbrega' 'custódio' 'natividade' 'flora' 'cláudia']
 ['ângela' 'eugênia' 'tomásia' 'melchior' 'helena' 'camargo' 'estácio' 'úrsula']
 ['procópio' 'valéria' 'jorge' 'garcia' 'madrasta' 'enteada' 'iaiá' 'estela']
 ['carmo' 'libertos' 'prainha' 'noronha' 'cesária' 'aguiar' 'fidélia' 'tristão']
 ['damasceno' 'eusébia' 'cubas' 'borba' 'sabina' 'cotrim' 'marcela' 'virgília']
 ['camacho' 'teófilo' 'tonica' 'borba' 'sofia' 'fernanda' 'benedita' 'rubião']
 ['gamaliel' 'dudu' 'eufrasina' 'tatá' 'belmira' 'tónico' 'zina' 'clarissa']
 ['campolargo' 'vacariano' 'vivaldino' 'getúl

## Dividindo o dataset entre treinamento e validação

In [7]:
nr.seed(20170607)

batch_size  = 32
seq_size    = 500
valid_split = 0.2

### Divisão simples

In [8]:
all_data = [[] for i in range(n_labels)]

for sequence, label in zip(sequences, book_label):
    n_seqs = len(sequence) // seq_size
    for i in range(n_seqs):
        beg = i * seq_size
        all_data[label].append(sequence[beg:beg+seq_size])

N = 10 * (min([len(x) for x in all_data]) // 10)
all_data = np.array([seq[:N] for seq in all_data], np.int32).reshape(-1, 500)
all_labels = np.array([[i] * N for i in range(n_labels)], np.int32).reshape(-1)
print(all_data.shape, all_labels.shape)


(2550, 500) (2550,)


In [9]:
from sklearn.model_selection import train_test_split

Xtra, Xval, ytra, yval = train_test_split(all_data, all_labels, test_size=valid_split)
print(Xtra.shape, ytra.shape, Xval.shape, yval.shape)

(2040, 500) (2040,) (510, 500) (510,)


In [11]:
fn = data_dir + 'livros_sequences.npz'
np.savez_compressed(fn, Xtra=Xtra, Xval=Xval, ytra=ytra, yval=yval)

### Divisão para uso com geradores e aumento de dados

In [12]:
valid_length = [int(0.2 * len(x)) for x in sequences]
valid_start = [nr.randint(2000, len(x) - 2000 - n) for x, n in zip(sequences, valid_length)]

valid_sequences = [seq[x0:x0+n] for seq, x0, n in zip(sequences, valid_start, valid_length)]

train_sequences = [seq[:x0] for seq, x0 in zip(sequences, valid_start)] + \
                  [seq[x0+n:] for seq, x0, n in zip(sequences, valid_start, valid_length)]

valid_labels = book_label
train_labels = book_label + book_label

n_train_words = sum([len(x) for x in train_sequences])
n_valid_words = sum([len(x) for x in valid_sequences])

print('Training sequences:')
print('-------------------')
for i, (seq, lab) in enumerate(zip(train_sequences, train_labels)):
    if i < n_books:
        print('{:2d}. {:16s} ({}) -- {:6d} palavras do início do livro {}'.format(i+1, book_author[i%n_books], lab,
                                                                                  len(seq), book_title[i%n_books]))
    else:
        print('{:2d}. {:16s} ({}) -- {:6d} palavras do final do livro  {}'.format(i+1, book_author[i%n_books], lab,
                                                                                  len(seq), book_title[i%n_books]))
print()
print('Validation sequences:')
print('---------------------')
for i, (seq, lab) in enumerate(zip(valid_sequences, valid_labels)):
    print('{:2d}. {:16s} ({}) -- {:6d} palavras do meio do livro {}'.format(i+1, book_author[i%n_books], lab,
                                                                            len(seq), book_title[i%n_books]))
print()
print('Total number of training words:  ', n_train_words)
print('Total number of validation words:', n_valid_words)

Training sequences:
-------------------
 1. Jorge_Amado      (2) --  44497 palavras do início do livro Capitães_de_Areia
 2. Jorge_Amado      (2) --  92458 palavras do início do livro Dona_flor_seus_dois_maridos
 3. Jorge_Amado      (2) --  86994 palavras do início do livro Gabriela
 4. Jorge_Amado      (2) --  49425 palavras do início do livro Tereza_Batista_Cansada_de_Guerra
 5. Machado_de_Assis (0) --  10722 palavras do início do livro Dom_Casmurro
 6. Machado_de_Assis (0) --  48702 palavras do início do livro Esaú_e_Jacó
 7. Machado_de_Assis (0) --  20724 palavras do início do livro Helena
 8. Machado_de_Assis (0) --  28329 palavras do início do livro Iaiá_Garcia
 9. Machado_de_Assis (0) --  33510 palavras do início do livro Memorial_de_Aires
10. Machado_de_Assis (0) --  26884 palavras do início do livro Memórias_Póstumas_de_Brás_Cubas
11. Machado_de_Assis (0) --  32579 palavras do início do livro Quincas_Borba
12. Érico_Veríssimo (1) --  21065 palavras do início do livro 

In [13]:
import pickle
fn = data_dir + 'livros_sequences_augm.pkl'
pickle.dump([train_sequences, valid_sequences, train_labels, valid_labels], open(fn, 'wb'))

### Criando  geradores para treino e validação

In [14]:
class MyDataGenerator:
    def __init__(self, batch_size, seq_size, sequences, labels):
        self.batch_size = batch_size
        self.length = seq_size
        self.sequences = sequences
        self.labels = labels
        sizes = np.array([len(seq) for seq in sequences])
        self.p = 1.0 * sizes / sizes.sum()        # probabilidade de escolha para cada sequencia
        self.n = np.arange(len(sequences))        # indices de cada sequencia (para o choice abaixo)
        
    def __call__(self):
        while True:
            batch = np.empty((self.batch_size, self.length), np.int32)
            label = np.empty((self.batch_size, n_labels), np.int32)
            for i in range(self.batch_size):
                k = nr.choice(self.n, p=self.p)
                p = nr.randint(0, len(self.sequences[k]) - self.length)
                batch[i] = self.sequences[k][p:p+self.length]
                label[i] = to_categorical(self.labels[k], num_classes=n_labels)
            yield batch, label

            
train_gen = MyDataGenerator(batch_size, seq_size, train_sequences, train_labels)()
valid_gen = MyDataGenerator(batch_size, seq_size, valid_sequences, valid_labels)()
